In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [6]:
prod = ['140A',
        '143A',
        '349A',
        '582A',
        '964A',]

In [7]:
test = ['029A',
        '980A',
        '179A',
        '456A',
        '379A',
       ]

In [8]:
from collections import deque
from functools import cache, lru_cache

# Numeric keypad layout
NUM_POS = {
    '7': (0,0), '8':(1,0), '9':(2,0),
    '4': (0,1), '5':(1,1), '6':(2,1),
    '1': (0,2), '2':(1,2), '3':(2,2),
    '0': (1,3), 'A':(2,3),
}
# Directional keypad layout
DIR_POS = {
    '^': (1,0), 'A':(2,0),
    '<': (0,1), 'v':(1,1), '>' :(2,1),
}


def build_distances(pos_map):
    dists = {}
    for src in pos_map:
        d = {}
        dist_steps ={}
        sx, sy = pos_map[src]
        queue = deque([(sx, sy, "", 0)])
        seen = {(sx, sy): 0}
        while queue:
            x, y, path,steps = queue.popleft()
            
            if (x,y) in pos_map.values():
                tgt = next(k for k,v in pos_map.items() if v==(x,y))
                if tgt not in d.keys() or dist_steps[tgt] == steps:
                    dist_steps[tgt] = steps
                    try:
                        d[tgt].append(path)
                    except KeyError:
                        d[tgt] = [path]
                    
            for move, (dx, dy) in {'<':(-1,0),'>':(1,0),'^':(0,-1),'v':(0,1)}.items():
                nx, ny = x+dx, y+dy
                new_steps = steps + 1
                if (nx, ny) in pos_map.values() and ( (nx,ny) not in seen.keys() or new_steps <= seen[(nx,ny)] ):
                    seen[(nx,ny)] = new_steps
                    queue.append((nx, ny, path + move, new_steps))
        dists[src] = d
    return dists

# Compute movement maps for both keypads
NUM_PATHS = build_distances(NUM_POS)
DIR_PATHS = build_distances(DIR_POS)

@cache
def generate_upper_sequence(code, numbers=False):
    shortest_paths = NUM_PATHS if numbers else DIR_PATHS
    upper_return=[]
        
    full_path = 'A' + code
    upper_candidates = ['']
    for s, e in zip(full_path[:-1], full_path[1:]):
        new_upper=[]
        for upper_candidate in upper_candidates:
            for upper_chunk in shortest_paths[s][e]:
                new_upper.append(upper_candidate+upper_chunk+'A')
        upper_candidates = new_upper.copy()
        
    return tuple(upper_candidates)

In [9]:
def find(string, char):
    return [i for i, ltr in enumerate(string) if ltr == char]

@cache
def recurse_segment(code: str, depth: int) -> str:
    """
    Recursively builds all possible minimal-length input sequences
    needed at layer `depth` to produce `code` at the bottom.
    """
    
    longer = 'A'+code
    alocs = find(longer,'A')

    total_len=0
    if depth == 0 : 
        for i,j in zip( alocs, alocs[1:]):
            lower_code = longer[i+1: j+1]
            next_segs =  generate_upper_sequence(lower_code) 
            total_len += min ([ len(ns) for ns in next_segs ])
        return total_len
        
    for i,j in zip( alocs, alocs[1:]):
        lower_code = longer[i+1: j+1]
        key = (lower_code, depth)
        
        next_segs =  generate_upper_sequence(lower_code) 
        
        total_len += min( [ recurse_segment(ns, depth-1) for ns in next_segs])
    return total_len

In [10]:
def complexity(codes, nrobots=25):
    complexity = 0
    for code in codes:
        # Clean and format the code
        clean_code = code.strip()
        if not clean_code.endswith('A'):
            clean_code += 'A'

        r1 = generate_upper_sequence(code, True)
        path_length = min( [ recurse_segment(r, nrobots-1) for r in r1])

        print(path_length,int(code.strip('A')))
        
        complexity += path_length*int(code.strip('A'))
    return complexity


In [11]:
%%time
complexity(test,2)

68 29
60 980
68 179
64 456
64 379
CPU times: user 1.15 ms, sys: 29 μs, total: 1.18 ms
Wall time: 1.19 ms


126384

In [12]:
%%time
complexity(prod,2)

70 140
72 143
72 349
68 582
72 964
CPU times: user 1.62 ms, sys: 173 μs, total: 1.8 ms
Wall time: 1.76 ms


154208

In [13]:
%%time
complexity(prod,25)

87513499934 140
89741193600 143
87793663956 349
86475783008 582
85006969638 964
CPU times: user 11.7 ms, sys: 1.85 ms, total: 13.6 ms
Wall time: 12 ms


188000493837892

# Part 2

In [90]:
def find(string, char):
    return [i for i, ltr in enumerate(string) if ltr == char]
    
def chunk_string(code):
    longer = 'A'+code
    alocs = find(longer,'A')
    upper=''
    for i,j in zip( alocs, alocs[1:]):
        upper = upper + generate_upper_sequence(longer[i+1: j+1])
    return upper 


#seen_code = {}
@cache
def recurse_segment(code: str, depth: int) -> str:
    """
    Recursively builds all possible minimal-length input sequences
    needed at layer `depth` to produce `code` at the bottom.
    """
    
    longer = 'A'+code
    alocs = find(longer,'A')

    total_len=0
    if depth == 0 : 
        for i,j in zip( alocs, alocs[1:]):
            lower_code = longer[i+1: j+1]
            next_seg =  generate_upper_sequence(lower_code) 
            total_len += len(next_seg)
        return total_len
        
    for i,j in zip( alocs, alocs[1:]):
        lower_code = longer[i+1: j+1]
        key = (lower_code, depth)
        
        next_seg =  generate_upper_sequence(lower_code) 
        
        total_len += recurse_segment(next_seg, depth-1)
    return total_len

In [91]:
def part_two_complexity(codes, nrobots=25):
    complexity = 0
    for code in codes:
        # Clean and format the code
        clean_code = code.strip()
        if not clean_code.endswith('A'):
            clean_code += 'A'

        r1 = generate_upper_sequence(code, True)
        path_length =recurse_segment(r1, nrobots-1)

        print(path_length,int(code.strip('A')))
        
        complexity += path_length*int(code.strip('A'))
    return complexity


In [92]:
%%time
part_two_complexity(test,2)

68 29
60 980
68 179
64 456
64 379
CPU times: user 333 μs, sys: 58 μs, total: 391 μs
Wall time: 391 μs


126384

In [35]:
%%time
part_two_complexity(prod,2)

70 140
72 143
72 349
68 582
72 964
CPU times: user 557 μs, sys: 19 μs, total: 576 μs
Wall time: 571 μs


154208

In [81]:
%%time
part_two_complexity(prod,25)

99333974668 140
101779364540 143
101257214304 349
98217357614 582
97169520992 964
CPU times: user 213 μs, sys: 45 μs, total: 258 μs
Wall time: 266 μs


214633893742472

In [82]:
%%time
part_two_complexity(prod,24)

39682956354 140
40659866182 143
40451274276 349
39236878176 582
38818276878 964
CPU times: user 401 μs, sys: 90 μs, total: 491 μs
Wall time: 471 μs


85744151484734

In [ ]:
# 214633893742472 -- too high

In [40]:
def simulate_robot_layer_output(code_sequence, keypad_layout, start='A'):
    # Build reverse map from position to key
    pos_to_key = {v: k for k, v in keypad_layout.items()}
    x, y = keypad_layout[start]
    output = ''
    for ch in code_sequence:
        if ch in {'<', '>', '^', 'v'}:
            dx, dy = {'<': (-1, 0), '>': (1, 0), '^': (0, -1), 'v': (0, 1)}[ch]
            nx, ny = x + dx, y + dy
            if (nx, ny) in pos_to_key:
                x, y = nx, ny
            else:
                raise ValueError(f"Invalid move {ch} from position {(x, y)}")
        elif ch == 'A':
            output += pos_to_key[(x, y)]
        else:
            raise ValueError(f"Invalid input character: {ch}")
    return output


In [42]:
simulate_robot_layer_output('<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A', DIR_POS)

'<A>Av<<AA>^AA>AvAA^A<vAAA>^A'

In [45]:
simulate_robot_layer_output('v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<A>vA^A', DIR_POS)

'<A>A<AAv<AA>>^AvAA^Av<AAA^>A'

In [59]:
start='<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A'
print(len(start), start)
tmp = simulate_robot_layer_output(start, DIR_POS)
print(len(tmp), tmp)
tmp = simulate_robot_layer_output(tmp, DIR_POS)
print(len(tmp), tmp)
simulate_robot_layer_output(tmp, NUM_POS)

64 <v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A
28 <A>Av<<AA>^AA>AvAA^A<vAAA>^A
14 ^A<<^^A>>AvvvA


'379A'

In [60]:
start='v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<A>vA^A'
print(len(start), start)
tmp = simulate_robot_layer_output('v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<A>vA^A', DIR_POS)
print(len(tmp), tmp)
tmp = simulate_robot_layer_output(tmp, DIR_POS)
print(len(tmp), tmp)
simulate_robot_layer_output(tmp, NUM_POS)

68 v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<A>vA^A
28 <A>A<AAv<AA>>^AvAA^Av<AAA^>A
14 ^A^^<<A>>AvvvA


'379A'

In [95]:
import sys
import re
from collections import deque, Counter, defaultdict
from pathlib import Path
from pprint import pprint
from functools import partial, lru_cache
from itertools import product, combinations

# Run with test data    -> python3 -m d#p#
# Run with puzzle data  -> python3 -m d#p# X (any argument)

def main(robots=25):

    # Puzzle code
    #----------------------------------------------------------------

    # Setup lookup maps
    codes = prod
    numpad = {
        "A": {"0": ("<A",),
              "1": ("<^<A", "^<<A",),
              "2": ("<^A", "^<A",),
              "3": ("^A",),
              "4": ("<^<^A", "<^^<A", "^<<^A", "^<^<A", "^^<<A",),
              "5": ("<^^A", "^<^A", "^^<A",),
              "6": ("^^A",),
              "7": ("<^<^^A", "<^^<^A", "<^^^<A", "^<<^^A", "^<^<^A", "^<^^<A",
                    "^^<<^A", "^^<^<A", "^^^<<A",),
              "8": ("<^^^A", "^<^^A", "^^<^A", "^^^<A",),
              "9": ("^^^A",)},
        "0": {"0": ("A",),
              "1": ("^<A",),
              "2": ("^A",),
              "3": ("^>A", ">^A",),
              "4": ("^<^A", "^^<A",),
              "5": ("^^A",),
              "6": ("^^>A", "^>^A", ">>^A",),
              "7": ("^<^^A", "^^<^A", "^^^<A",),
              "8": ("^^^A",),
              "9": ("^^^>A", "^^>^A", "^>^^A", ">^^^A",)},
        "1": {"1": ("A",),
              "2": (">A",),
              "3": (">>A",),
              "4": ("^A",),
              "5": ("^>A", ">^A",),
              "6": ("^>>A", ">^>A", ">>^A",),
              "7": ("^^A",),
              "8": ("^^>A", "^>^A", ">>^A",),
              "9": ("^^>>A", "^>^>A", "^>>^A", ">^^>A", ">^>^A", ">>^^A",)},
        "2": {"2": ("A",),
              "3": (">A",),
              "4": ("<^A", "^<A",),
              "5": ("^A",),
              "6": ("^>A", ">^A",),
              "7": ("<^^A", "^<^A", "^^<A",),
              "8": ("^^A",),
              "9": ("^^>A", "^>^A", ">^^A",)},
        "3": {"3": ("A",),
              "4": ("<<^A", "<^<A", "^<<A",),
              "5": ("<^A", "^<A",),
              "6": ("^A",),
              "7": ("<<^^A", "<^<^A", "<^^<A", "^<<^A", "^<^<A", "^^<<A",),
              "8": ("<^^A", "^<^A", "^^<A",),
              "9": ("^^A",)},
        "4": {"4": ("A",),
              "5": (">A",),
              "6": (">>A",),
              "7": ("^A",),
              "8": ("^>A", ">^A",),
              "9": ("^>>A", ">^>A", ">>^A",)},
        "5": {"5": ("A",),
              "6": (">A",),
              "7": ("<^A", "^<A",),
              "8": ("^A",),
              "9": ("^>A", ">^A",)},
        "6": {"6": ("A",),
              "7": ("<<^A", "<^<A", "^<<A",),
              "8": ("<^A", "^<A",),
              "9": ("^A",)},
        "7": {"7": ("A",),
              "8": (">A",),
              "9": (">>A",)},
        "8": {"8": ("A",),
              "9": (">A",)},
        "9": {"9": ("A",),},
    }

    # Start recurseive search with memoization
    human_at_level = robots + 1
    ans = 0
    for code in codes:
        prev_button = "A"
        final_length = 0
        for next_button in code:
            shortest_sequence = 10 ** 20
            try:
                possible_paths = numpad[prev_button][next_button]
            except KeyError:
                possible_paths = reverse_sequences(numpad[next_button][prev_button])    
            for path in possible_paths:
                shortest_sequence = min(shortest_sequence, find_shortest(path, robot_level=1, human_level=human_at_level))
            final_length += shortest_sequence
            prev_button = next_button

        print(f"{code} {final_length=}")
        ans += final_length * int(code[:-1])

    print(f"Ans: {ans}")
    # ans: 189235298434780

@lru_cache
def reverse_sequences(directions: tuple[str]) -> tuple[str]:
    reversal_map = {
        "^": "v",
        ">": "<",
        "v": "^",
        "<": ">",
    }
    
    reversed_possibilities = tuple()
    for direction in directions:
        rev = ""
        for c in direction[-2::-1]:
            rev += reversal_map[c]
        reversed_possibilities += (rev+"A",)
    return reversed_possibilities

@lru_cache
def find_shortest(path: str, robot_level: int, human_level: int) -> int:
    if robot_level == human_level:
        return len(path)

    prev_button = "A"
    final_length = 0
    for next_button in path:
        shortest_sequence = 10 ** 20
        possible_paths = get_dirpad_paths(prev_button, next_button)
   
        for path in possible_paths:
            shortest_sequence = min(shortest_sequence, find_shortest(path, robot_level=robot_level+1, human_level=human_level))
        final_length += shortest_sequence
        prev_button = next_button
    return final_length

@lru_cache
def get_dirpad_paths(from_key: str, to_key: str) -> tuple[str]:
    dirpad = {
        "A": {"A": ("A",),
              "^": ("<A",),
              "<": ("<v<A", "v<<A",),
              "v": ("<vA", "v<A",),
              ">": ("vA",)},
        "^": {"^": ("A",),
              "<": ("v<A",),
              "v": ("vA",),
              ">": ("v>A", ">vA",)},
        "<": {"<": ("A",),
              "v": (">A",),
              ">": (">>A",)},
        "v": {"v": ("A",),
              ">": (">A",)},
        ">": {">": ("A",),},
    }
    try:
        possible_paths = dirpad[from_key][to_key]
    except KeyError:
        possible_paths = reverse_sequences(dirpad[to_key][from_key])
    return possible_paths





In [96]:
main()

140A final_length=87513499934
143A final_length=89741193600
349A final_length=87793663956
582A final_length=86475783008
964A final_length=85006969638
Ans: 188000493837892


In [104]:
main(4)

140A final_length=430
143A final_length=442
349A final_length=432
582A final_length=424
964A final_length=422
Ans: 927750


In [105]:

part_two_complexity(prod,4)


430 140
442 143
438 349
424 582
422 964


929844

In [94]:
188000493837892 -214633893742472

-26633399904580

In [5]:
complexity(prod,25)

NameError: name 'complexity' is not defined